In [61]:
import os
import re
import json

import pandas as pd

In [62]:
WIKI_URL = r"https://en.wikipedia.org/wiki/List_of_MythBusters_episodes"
FILENAME = os.path.abspath(os.path.join(os.path.dirname(__name__), "src", "mythbusters.json"))

In [63]:
def get_season(season_table, index):
    season = season_table.loc[index]["Season"].values[1]
    return f"{season} season"

In [64]:
def normalize(name):
    name = name.lower()
    name = name.replace(" / ", " ").replace("/", " ").replace(" - ", "-")
    name = re.sub(r"[^-\w\s]", "", name)
    name = name.replace(" ", "-")
    return name

In [65]:
def process_table(df, season):
    episodes = []
    for index, row in df[::2].iterrows():
        for name in re.findall(r"\"(.+?)\"", row["Title"]):
            episodes.append(
                {
                    "name": normalize(name),
                    "episode": row["No. in season"],
                    "season": season,
                }
            )
    return episodes

In [66]:
df = pd.read_html(WIKI_URL)
len(df)

22

In [67]:
results = []
for ind, table in enumerate(df):
    if "Title" in table:
        season = get_season(df[0], ind - 1)
        episodes = process_table(table, season)
        results.extend(episodes)

In [68]:
with open(FILENAME, "w") as f:
    json.dump(results, f)